## A2A + MCP + LangChain = Powerful Multi-Agent Chatbot!
### We're building a multi-agent financial analysis system where different AI agents work together to provide comprehensive stock market insights.

### Install all required libraries

In [16]:
!pip install python-a2a fastmcp langchain-openai yfinance pandas requests beautifulsoup4 --quiet

In [17]:
import os
import re
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from python_a2a import OpenAIA2AServer, run_server
from python_a2a import AgentCard, AgentSkill
from python_a2a.mcp import FastMCP
from python_a2a.langchain import to_langchain_agent, to_langchain_tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# Set your OpenAI API key directly here
OPENAI_API_KEY = "Add Your OpenAI API Key"  # Replace with your actual key
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Define the financial expert agent's capabilities

In [18]:
# Define the financial expert agent's capabilities
agent_card = AgentCard(
    name="Stock Market Expert",
    description="Expert in market trends, fundamentals, and investment strategies.",
    url="http://localhost:5000",
    version="1.0.0",
    skills=[
        AgentSkill(
            name="Market Analysis",
            description="Analyze overall market sentiment and key indicators.",
            examples=["What's the current market sentiment?", "Impact of interest rates on tech stocks?"]
        ),
        AgentSkill(
            name="Investment Strategies",
            description="Describe risk management and portfolio diversification.",
            examples=["How to diversify my portfolio?", "Explanation of dollar-cost averaging."]
        ),
        AgentSkill(
            name="Company Analysis",
            description="Interpret financial ratios and company fundamentals.",
            examples=["How to read P/E ratios?", "Key metrics for evaluating growth stocks."]
        )
    ]
)

print("Agent profile created successfully!")
print(f"Agent Name: {agent_card.name}")
print(f"Skills: {[skill.name for skill in agent_card.skills]}")

Agent profile created successfully!
Agent Name: Stock Market Expert
Skills: ['Market Analysis', 'Investment Strategies', 'Company Analysis']


### Create the A2A server instance

In [19]:
# Create the A2A server instance
a2a_server = OpenAIA2AServer(
    api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0.0,
    system_prompt="You are a stock market and financial analysis expert. Provide factual, concise insights based on available data and market knowledge."
)

print("A2A Server initialized successfully!")

A2A Server initialized successfully!


In [20]:
import threading
import time

def start_a2a_server():
    """Start A2A server in background thread"""
    try:
        run_server(a2a_server, host='0.0.0.0', port=5000)
    except Exception as e:
        print(f"A2A Server error: {e}")

# Start server in background thread
a2a_thread = threading.Thread(target=start_a2a_server, daemon=True)
a2a_thread.start()

# Give server time to start
time.sleep(3)
print("A2A Server started on http://localhost:5000")

Starting A2A server on http://0.0.0.0:5000/a2a
 * Serving Flask app 'python_a2a.server.http' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
A2A Server error: [Errno 98] Address already in use
A2A Server started on http://localhost:5000


### Create MCP server for financial tools

In [21]:
# Create MCP server for financial tools
mcp_server = FastMCP(
    name="FinanceTools",
    description="Tools for retrieving stock data and financial news."
)

print("MCP Server initialized!")

INFO:python_a2a.mcp.fastmcp:Initialized FastMCP server: FinanceTools v1.0.0


MCP Server initialized!


In [22]:
@mcp_server.tool(
    name="stock_data",
    description="Fetch current metrics and data for stocks by ticker symbols or company names."
)
def stock_data(input_str=None, **kwargs):
    """Fetch stock data using yfinance"""
    input_str = kwargs.get('input', input_str)
    if not input_str:
        return {"error": "No input provided."}
    
    # Extract ticker symbols
    tickers = []
    if ',' in input_str:
        tickers = [t.strip().upper() for t in input_str.split(',')]
    else:
        tickers = [w.upper() for w in re.findall(r"\b[A-Za-z]{1,5}\b", input_str)]
    
    # Handle common company names
    common_names = {
        'apple': 'AAPL', 
        'nvidia': 'NVDA', 
        'microsoft': 'MSFT',
        'google': 'GOOGL',
        'amazon': 'AMZN',
        'tesla': 'TSLA'
    }
    
    if not tickers:
        for name, ticker in common_names.items():
            if name in input_str.lower(): 
                tickers.append(ticker)
    
    results = {}
    for ticker in tickers:
        try:
            tk = yfinance.Ticker(ticker)
            hist = tk.history(period="1mo")
            
            if hist.empty:
                results[ticker] = {"error": "No data available."}
                continue
            
            first_day = hist.iloc[0]
            last_day = hist.iloc[-1]
            price_change = float(last_day['Close'] - first_day['Close'])
            pct_change = price_change / float(first_day['Close']) * 100
            
            info = tk.info
            
            summary = {
                "latest_price": float(last_day['Close']),
                "price_change": price_change,
                "percent_change": pct_change,
                "52_week_high": info.get('fiftyTwoWeekHigh'),
                "52_week_low": info.get('fiftyTwoWeekLow'),
                "market_cap": info.get('marketCap'),
                "pe_ratio": info.get('trailingPE'),
                "volume": int(last_day['Volume'])
            }
            results[ticker] = summary
            
        except Exception as e:
            results[ticker] = {"error": f"Failed to fetch data: {str(e)}"}
    
    return results

print("Stock data fetcher tool created!")

INFO:python_a2a.mcp.fastmcp:Registered tool: stock_data


Stock data fetcher tool created!


In [23]:
@mcp_server.tool(
    name="web_scraper",
    description="Scrape latest financial headlines and company snapshot from Finviz."
)
def web_scraper(input_str=None, **kwargs):
    """Scrape financial news and data from Finviz"""
    ticker = (kwargs.get('input') or input_str or '').strip().upper()
    
    if not ticker:
        return {"error": "No ticker symbol provided."}
    
    url = f"https://finviz.com/quote.ashx?t={ticker.lower()}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract news items
        news_items = []
        news_table = soup.find('table', {'id': 'news-table'})
        if news_table:
            for row in news_table.find_all('tr')[:5]:  # Get top 5 news items
                cells = row.find_all('td')
                if len(cells) >= 2:
                    date_cell = cells[0]
                    title_cell = cells[1]
                    
                    link_tag = title_cell.find('a')
                    if link_tag:
                        title = link_tag.text.strip()
                        link = link_tag.get('href', '')
                        if link and not link.startswith('http'):
                            link = urljoin(url, link)
                        
                        news_items.append({
                            "date": date_cell.text.strip(),
                            "title": title,
                            "link": link
                        })
        
        # Extract company snapshot data
        snapshot_details = {}
        snapshot_table = soup.find('table', {'class': 'snapshot-table2'})
        if snapshot_table:
            for row in snapshot_table.find_all('tr'):
                cells = row.find_all('td')
                for i in range(0, len(cells), 2):
                    if i + 1 < len(cells):
                        key = cells[i].text.strip()
                        value = cells[i + 1].text.strip()
                        snapshot_details[key] = value
        
        return {
            "ticker": ticker,
            "news_items": news_items,
            "snapshot": snapshot_details
        }
        
    except Exception as e:
        return {"error": f"Failed to scrape data: {str(e)}"}

print("Financial news scraper tool created!")

INFO:python_a2a.mcp.fastmcp:Registered tool: web_scraper


Financial news scraper tool created!


In [24]:
def start_mcp_server():
    """Start MCP server in background thread"""
    try:
        mcp_server.run(host='0.0.0.0', port=6000)
    except Exception as e:
        print(f"MCP Server error: {e}")

# Start MCP server in background thread
mcp_thread = threading.Thread(target=start_mcp_server, daemon=True)
mcp_thread.start()

# Give server time to start
time.sleep(3)
print("MCP Server started on http://localhost:6000")

INFO:     Started server process [220]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 6000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


MCP Server started on http://localhost:6000


### Convert both A2A agent & MCP tools to LangChain

In [25]:
# Convert A2A agent to LangChain
try:
    a2a_agent = to_langchain_agent("http://localhost:5000")
    print("A2A agent converted to LangChain successfully!")
except Exception as e:
    print(f"Error converting A2A agent: {e}")

# Convert MCP tools to LangChain
try:
    stock_tool = to_langchain_tool("http://localhost:6000", "stock_data")
    news_tool = to_langchain_tool("http://localhost:6000", "web_scraper")
    print("MCP tools converted to LangChain successfully!")
except Exception as e:
    print(f"Error converting MCP tools: {e}")

INFO:python_a2a.langchain.mcp:Found 2 tools on MCP server
INFO:python_a2a.langchain.mcp:Creating LangChain tool for MCP tool: stock_data
INFO:python_a2a.langchain.mcp:Successfully created LangChain tool: stock_data
INFO:python_a2a.langchain.mcp:Found 2 tools on MCP server
INFO:python_a2a.langchain.mcp:Creating LangChain tool for MCP tool: web_scraper
INFO:python_a2a.langchain.mcp:Successfully created LangChain tool: web_scraper


A2A agent converted to LangChain successfully!
MCP tools converted to LangChain successfully!


In [26]:
def ask_expert(query):
    """Ask the financial expert agent"""
    try:
        result = a2a_agent.invoke(query)
        return result.get('output', 'No response from expert')
    except Exception as e:
        return f"Error asking expert: {str(e)}"

def fetch_stock_data(query):
    """Fetch stock data"""
    try:
        return stock_tool.invoke(query)
    except Exception as e:
        return f"Error fetching stock data: {str(e)}"

def fetch_financial_news(query):
    """Fetch financial news"""
    try:
        return news_tool.invoke(query)
    except Exception as e:
        return f"Error fetching news: {str(e)}"

print("Wrapper functions defined!")

Wrapper functions defined!


### Create LangChain Tool objects

In [27]:
# Create LangChain Tool objects
tools = [
    Tool(
        name="StockExpert",
        func=ask_expert,
        description="Ask financial questions to a stock market expert. Use for market analysis, investment strategies, and financial advice."
    ),
    Tool(
        name="StockData",
        func=fetch_stock_data,
        description="Retrieve current stock metrics and data. Input should be ticker symbols or company names."
    ),
    Tool(
        name="FinancialNews",
        func=fetch_financial_news,
        description="Get latest financial headlines and company snapshots. Input should be a ticker symbol."
    )
]

print(f"Created {len(tools)} LangChain tools!")

Created 3 LangChain tools!


### Initialize the main LLM & meta-agent

In [28]:
# Initialize the main LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    api_key=OPENAI_API_KEY
)

# Create the meta-agent
meta_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate"
)

print("Meta-agent initialized successfully!")

Meta-agent initialized successfully!


/tmp/ipykernel_220/2833923176.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  meta_agent = initialize_agent(


### Test both Test stock data fetcher & news scraper

In [29]:
# Test stock data fetcher
print("Testing stock data fetcher...")
test_stock_data = stock_data("AAPL,NVDA")
print("Stock Data Result:", test_stock_data)
print("\n" + "="*50 + "\n")

# Test news scraper
print("Testing news scraper...")
test_news_data = web_scraper("AAPL")
print("News Data Result:", test_news_data)
print("\n" + "="*50 + "\n")

Testing stock data fetcher...
Stock Data Result: {'AAPL': {'latest_price': 200.85000610351562, 'price_change': -11.3717041015625, 'percent_change': -5.358407530772218, '52_week_high': 260.1, '52_week_low': 169.21, 'market_cap': 2999855611904, 'pe_ratio': 31.236393, 'volume': 70753100}, 'NVDA': {'latest_price': 135.1300048828125, 'price_change': 26.210006713867188, 'percent_change': 24.06353943672765, '52_week_high': 153.13, '52_week_low': 86.62, 'market_cap': 3295496634368, 'pe_ratio': 43.731396, 'volume': 332443400}}


Testing news scraper...
News Data Result: {'ticker': 'AAPL', 'news_items': [{'date': 'Jun-01-25 11:52PM', 'title': "Xiaomi's XRing O1 fast-tracks in-house SoC lessons Apple took 15 years to learn", 'link': 'https://www.digitimes.com/news/a20250602PD215/xiaomi-apple-soc-ceo-samsung.html'}, {'date': '08:06PM', 'title': 'Stock market today: Dow, S&P 500, Nasdaq futures slip as Wall Street kicks off June after bullish May', 'link': 'https://finance.yahoo.com/news/live/stock

### Test various types of queries

In [30]:
# Test various types of queries
test_queries = [
    "What's the P/E ratio of Tesla and should I invest in it?",
    "Compare the market performance of Apple and Microsoft this month",
    "What are the top financial news headlines for NVDA?",
    "Explain the concept of dollar-cost averaging and its benefits"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)
    
    try:
        response = meta_agent.invoke(query)
        print(response['output'])
    except Exception as e:
        print(f"Error: {e}")
    
    print('='*60)


TEST QUERY 1: What's the P/E ratio of Tesla and should I invest in it?


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `StockData` with `TSLA`


{"TSLA": {"latest_price": 346.4599914550781, "price_change": 64.29998779296875, "percent_change": 22.788484178632526, "52_week_high": 488.54, "52_week_low": 167.41, "market_cap": 1115933769728, "pe_ratio": 195.74011, "volume": 123097900}}

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `StockExpert` with `Tesla's P/E ratio is 195.74. Should I invest in it?`


A P/E (price-to-earnings) ratio of 195.74 indicates that Tesla's stock is trading at a high multiple of its earnings. This suggests that investors are willing to pay a premium for Tesla's future growth prospects. Here are some factors to consider before making an investment decision:

1. **Growth Potential**: Tesla is often valued for its potential growth in the electric vehicle market, energy storage, and other innovative technologies. Consider whether you believe Tesla can continue to grow at a pace that justifies its high P/E ratio.

2. **Market Position**: Tesla is a leader in the electric vehicle industry, with a strong brand and significant market share. Evaluate how well-positioned it is to maintain or expand its market position.

3. **Financial Health**: Review Tesla's financial statements to assess its revenue growth, profitability, and cash flow. A high P/E ratio can be more justifiable if t

...


Invoking: `FinancialNews` with `NVDA`


{"ticker": "NVDA", "news_items": [{"date": "Jun-01-25 11:43PM", "title": "Nvidia revenue jumps 69% despite China chip ban costs", "link": "https://www.digitimes.com/news/a20250602PD216/nvidia-revenue-guidance-market-growth.html"}, {"date": "11:12PM", "title": "Dow Jones Futures Fall Amid New Trump Tariffs; Palantir In Buy Zone As Tesla, Nvidia Fall Back", "link": "https://www.investors.com/market-trend/stock-market-today/dow-jones-futures-trump-tariff-news-palantir-tesla-nvidia-broadcom?mod=IBD_FV"}, {"date": "11:10PM", "title": "Foxconn-Nvidia to build Taiwan's 100MW AI supercomputing center", "link": "https://www.digitimes.com/news/a20250602PD205/taiwan-foxconn-nvidia-alliance-infrastructure.html"}, {"date": "09:54PM", "title": "Weekly news roundup: Huawei's 5nm PC, Wolfspeed's SiC crisis, and China's export chokehold", "link": "https://www.digitimes.com/news/a20250602VL204/huawei-wolfspeed-5nm-pc-sic-weekly-news-roundup.html"}, {"date": "08:0

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here are the top financial news headlines for Nvidia (NVDA):

1. **[Nvidia revenue jumps 69% despite China chip ban costs](https://www.digitimes.com/news/a20250602PD216/nvidia-revenue-guidance-market-growth.html)** - Jun-01-25 11:43PM
2. **[Dow Jones Futures Fall Amid New Trump Tariffs; Palantir In Buy Zone As Tesla, Nvidia Fall Back](https://www.investors.com/market-trend/stock-market-today/dow-jones-futures-trump-tariff-news-palantir-tesla-nvidia-broadcom?mod=IBD_FV)** - 11:12PM
3. **[Foxconn-Nvidia to build Taiwan's 100MW AI supercomputing center](https://www.digitimes.com/news/a20250602PD205/taiwan-foxconn-nvidia-alliance-infrastructure.html)** - 11:10PM
4. **[Weekly news roundup: Huawei's 5nm PC, Wolfspeed's SiC crisis, and China's export chokehold](https://www.digitimes.com/news/a20250602VL204/huawei-wolfspeed-5nm-pc-sic-weekly-news-roundup.html)** - 09:54PM
5. **[Stock market today: Dow, S&P 500, Nasdaq futures slip as Wall Street kicks off June after bullish May](https://financ

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Dollar-cost averaging (DCA) is an investment strategy where an investor divides the total amount to be invested across periodic purchases of a target asset, such as stocks or mutual funds. The purchases occur at regular intervals, regardless of the asset's price. This approach is designed to reduce the impact of volatility on the overall purchase. By spreading out the investment, the investor buys more shares when prices are low and fewer shares when prices are high.

### Benefits of Dollar-Cost Averaging:

1. **Reduces Impact of Volatility**: By investing a fixed amount regularly, investors can mitigate the effects of market volatility. This is because they are not trying to time the market, which can be unpredictable.

2. **Disciplined Investing**: DCA encourages a disciplined approach to investing. Investors commit to investing a set amount regularly, which can help in building a habit of saving and investing.

3. **Avoids Market Timing**: Since DCA involves investing at regular int

In [34]:
# Test various types of queries
test_queries = [
    "Should I invest in Google or Microsoft or Tesla stocks right now?",
    "What are some trending stocks & news from S&P?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)
    
    try:
        response = meta_agent.invoke(query)
        print(response['output'])
    except Exception as e:
        print(f"Error: {e}")
    
    print('='*60)


TEST QUERY 1: Should I invest in Google or Microsoft or Tesla stocks right now?


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `StockExpert` with `Should I invest in Google, Microsoft, or Tesla stocks right now?`


As of my last update in October 2023, I can't provide real-time investment advice, but I can offer some insights based on historical performance and general market trends for Google (Alphabet), Microsoft, and Tesla.

1. **Google (Alphabet Inc.)**:
   - **Strengths**: Alphabet has a strong position in digital advertising, cloud computing, and is investing heavily in AI and other emerging technologies. Its diverse revenue streams and strong balance sheet make it a relatively stable investment.
   - **Considerations**: Regulatory scrutiny and competition in digital advertising and cloud services are potential risks.

2. **Microsoft**:
   - **Strengths**: Microsoft has a robust cloud business with Azure, a strong presence in enterprise software, and is expanding in AI and gaming. Its consistent revenue growth and strong dividend policy make it attractive to many investors.
   - **Consideratio

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Investing in Google (Alphabet), Microsoft, or Tesla depends on your financial goals, risk tolerance, and investment horizon. Here's a brief overview of each:

1. **Google (Alphabet Inc.)**:
   - **Strengths**: Strong in digital advertising, cloud computing, and AI investments. It has diverse revenue streams and a solid balance sheet.
   - **Risks**: Faces regulatory scrutiny and competition in digital advertising and cloud services.

2. **Microsoft**:
   - **Strengths**: Robust cloud business with Azure, strong in enterprise software, and expanding in AI and gaming. Known for consistent revenue growth and a strong dividend policy.
   - **Risks**: Competition in cloud computing and potential regulatory challenges.

3. **Tesla**:
   - **Strengths**: Leader in electric vehicles (EVs) with a strong brand and innovative edge. Expanding into energy products and autonomous driving technology.
   - **Risks**: Stock is often volatile, faces competition from traditional automakers and new EV ent

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `FinancialNews` with `S&P 500`


{"ticker": "S&P 500", "news_items": [{"date": "May-30-25 10:27AM", "title": "Should You Buy, Sell or Hold CrowdStrike Stock Before Q1 Earnings?", "link": "https://finviz.com/news/70456/should-you-buy-sell-or-hold-crowdstrike-stock-before-q1-earnings"}, {"date": "May-29-25 01:40PM", "title": "SentinelOne's Profitability And Growth Trajectory Less Promising Than Competitors: Analyst", "link": "https://finance.yahoo.com/news/sentinelones-profitability-growth-trajectory-less-174022988.html"}, {"date": "01:13PM", "title": "Outage disrupts some SentinelOne services", "link": "https://finance.yahoo.com/m/1b7e7363-bcce-397e-b067-84f7b84a0c6f/outage-disrupts-some.html"}, {"date": "12:49PM", "title": "Why SentinelOne (S) Shares Are Getting Obliterated Today", "link": "https://finviz.com/news/69404/why-sentinelone-s-shares-are-getting-obliterated-today"}, {"date": "12:08PM", "title": "Chip stocks react to Nvidia, Boeing 737 Max, SentinelOne outlook", "l

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here are some of the latest trending news and updates related to the S&P 500:

1. **[Should You Buy, Sell or Hold CrowdStrike Stock Before Q1 Earnings?](https://finviz.com/news/70456/should-you-buy-sell-or-hold-crowdstrike-stock-before-q1-earnings)** - Published on May 30, 2025, at 10:27 AM.

2. **[SentinelOne's Profitability And Growth Trajectory Less Promising Than Competitors: Analyst](https://finance.yahoo.com/news/sentinelones-profitability-growth-trajectory-less-174022988.html)** - Published on May 29, 2025, at 01:40 PM.

3. **[Outage disrupts some SentinelOne services](https://finance.yahoo.com/m/1b7e7363-bcce-397e-b067-84f7b84a0c6f/outage-disrupts-some.html)** - Published at 01:13 PM.

4. **[Why SentinelOne (S) Shares Are Getting Obliterated Today](https://finviz.com/news/69404/why-sentinelone-s-shares-are-getting-obliterated-today)** - Published at 12:49 PM.

5. **[Chip stocks react to Nvidia, Boeing 737 Max, SentinelOne outlook](https://finance.yahoo.com/video/chip-stocks-rea